In [242]:
import keras
import pandas as pd
from datetime import datetime
import numpy as np
from matplotlib import style
import matplotlib.pyplot as plt
import yfinance as yf
import warnings
warnings.filterwarnings('ignore')
from sklearn.preprocessing import MinMaxScaler
from keras.models import Sequential
from keras.layers import Dense, LSTM
import joblib

In [243]:
df = yf.download("6920.T",start='2000-01-01',end = datetime.now(),interval="1d")

#　Closeコラムのみ抽出
data = df["Close"]
dataset = data.values

[*********************100%***********************]  1 of 1 completed


In [244]:
scaler = MinMaxScaler(feature_range=(0,1))
scaled_data = scaler.fit_transform(dataset)

In [245]:
#訓練データの取得
x_train = []
y_train = []
for i in range(120, len(scaled_data)): 
    x_train.append(scaled_data[i-120:i,0])
    y_train.append(scaled_data[i,0])

# 訓練データのreshape
x_train, y_train = np.array(x_train), np.array(y_train)
x_train = np.reshape(x_train,(x_train.shape[0], x_train.shape[1], 1))

In [ ]:
#LSTMモデル構築
model = Sequential()
model.add(LSTM(128,return_sequences = True, input_shape=(x_train.shape[1], 1)))
model.add(LSTM(64, return_sequences = False))
model.add(Dense(25))
model.add(Dense(1))
model.compile(optimizer='adam',loss='mean_squared_error')

#訓練用モデル構築
model.fit(x_train, y_train, epochs=1, batch_size=64)

55/55 ━━━━━━━━━━━━━━━━━━━━ 29s 453ms/step - loss: 0.0246


In [249]:
# モデルの保存
model.save('lstm_model.h5')

# scalerの保存
joblib.dump(scaler, 'scaler.save')

['scaler.save']